In [43]:
# #### Pre Installs
# import os
# os.environ['openAIAPIKey'] = ''
# os.environ['serpAPIKey'] = ''
!pip install google-search-results

In [33]:
appConfig = {
    "NoOfLinks" : 3,
    "oaSystemprompt" : "You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.",
    "oaUserprompt" : ""
}

In [34]:
import os

class getApiKeys:

  def __init__(self):
    pass

  def getApiKeys(self,apiKey):
    return os.environ.get(apiKey)

In [36]:
import serpapi
import json
from serpapi import GoogleSearch # Add this import statement

class searchForLinks:
  def __init__(self):
    pass

  def getSearchResults(self,searchWord):
    searpApiKey = getApiKeys()
    params = {
      "q": searchWord,
      "hl": "en",
      "gl": "us",
      "google_domain": "google.com",
      "api_key": searpApiKey.getApiKeys('serpAPIKey')
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    linksList = []

    if 'organic_results' in results:
        for res in results['organic_results']:
            if 'link' in res:  # Check if 'link' key exists in each result dictionary
                linksList.append(res['link'])
    else:
        print("No organic results found in the response.")

    return linksList[:appConfig['NoOfLinks']]

In [37]:
sfl = searchForLinks()
sfl.getSearchResults('coffee')

['https://en.wikipedia.org/wiki/Coffee',
 'https://bluebottlecoffee.com/?srsltid=AfmBOoq8gnRau5HYxQYISjN2ayTy3oODuySShGu6HG4dXRIOxORDg1dQ',
 'https://www.starbucks.com/']

In [62]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display


class webCrawler:

  def __init__(self):
    pass

  def crawlresults(self,srch):
    sfl = searchForLinks()
    linktocrawl = sfl.getSearchResults(srch)

    crawlJson = {}
    for links in linktocrawl:
      crawlJson.update(self.crawl(links))

    return crawlJson

  def crawl(self, url: str):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.title.string if soup.title else "Title not found"

    if soup.body:
      for irrelevant in soup.body(["script", "style","img","input"]):
        irrelevant.decompose()
      text = soup.body.get_text(separator="\n", strip=True)
    else:
      text = ""

    return {title : text}

    # links = [link.get('href') for link in soup.find_all('a')]
    # self.links = [link for link in links if link]

In [59]:
import os
import requests
# from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import time


class summarizer:

  def __init__(self):
    pass

  def user_prompt_for(self,title,text):
      user_prompt = f"You are looking at a website titled {title}"
      user_prompt += "The contents of this website is as follows; \
                      please find different types of coffees from  this website in markdown. \
                      If it includes news or announcements, then summarize these too.\n\n"
      user_prompt += text
      return user_prompt

  def messages_for(self,content):
      return [
          {"role": "system", "content": appConfig['oaSystemprompt']},
          {"role": "user", "content": content}
      ]

  def summarize(self,url):
      searpApiKey = getApiKeys()
      api_key = searpApiKey.getApiKeys('openAIAPIKey')

      openai = OpenAI(api_key=api_key)

      crawl = webCrawler()
      contents = crawl.crawlresults(url)

      responses = []

      for content in contents:
        user_prompt = self.user_prompt_for(content,contents[content])
        messages = self.messages_for(content)

        response = openai.chat.completions.create(
            model = "gpt-4o-mini",
            messages = messages
        )
        responses.append(response.choices[0].message.content)

      return responses


In [63]:
# summarize("coffee")
extract = summarizer()
results = extract.summarize('coffee')
for I in results:
  display(Markdown(I))

## Summary of Coffee - Wikipedia

Coffee is a popular brewed beverage made from roasted coffee beans, the seeds of berries from the Coffea plant. It is consumed around the world and is notable for its stimulating effect, largely attributed to caffeine. 

The history of coffee dates back to its discovery in Ethiopia, spreading to the Arabian Peninsula and then to the rest of the world. Coffee cultivation and trade have significantly shaped economies and cultures.

There are many different types of coffee drinks, including espresso, cappuccino, and latte, each varying in preparation methods and flavor profiles. The beverage is often enjoyed in social settings, and coffeehouses have historically served as important cultural hubs.

Nutritionally, coffee contains various antioxidants and has been linked to numerous health benefits, although excessive consumption can lead to negative side effects.

Coffee production involves several stages, including harvesting, processing, roasting, and brewing. Major coffee-producing countries include Brazil, Vietnam, and Colombia.

Sustainability and the impact of coffee farming on the environment is a growing concern, prompting initiatives aimed at promoting fair trading practices and environmentally friendly cultivation methods. 

Overall, coffee has a rich cultural significance and continues to be a crucial part of daily life for many people globally.

Peet's Coffee is a specialty coffee roaster and retailer established in 1966. The brand prides itself on crafting high-quality coffee with a focus on sourcing premium beans and offering a variety of blends and single-origin coffees. Peet's Coffee emphasizes its commitment to freshness, flavor, and sustainability, showcasing a strong heritage in the coffee industry. The company also offers a selection of teas and coffee-related merchandise, catering to a diverse customer base through its cafes and online store.

### Starbucks Coffee Company Summary

Starbucks Coffee Company is a global coffeehouse chain known for its specialty coffee drinks, teas, and a variety of food items. The company was founded in 1971 in Seattle, Washington, and has since grown to thousands of locations worldwide.

**Key Offerings:**
- **Beverages:** A wide range of coffee and espresso drinks, teas, and seasonal offerings.
- **Food:** Pastries, sandwiches, and snacks that complement their beverages.
- **Merchandise:** Coffee beans, brewing equipment, and branded merchandise are also available.

**Sustainability Initiatives:**
Starbucks emphasizes ethical sourcing of coffee, with programs focused on environmental stewardship and community support for coffee farmers.

**Customer Experience:**
The company prioritizes creating a welcoming atmosphere in their stores, often featuring comfortable seating and free Wi-Fi. They also have a rewards program that incentivizes customer loyalty through points for purchases.

**Innovations:**
Starbucks is known for integrating technology into the customer experience, including mobile ordering and payment options, to enhance convenience.

Overall, Starbucks is recognized not only for its products but also for its commitment to social responsibility and creating a distinctive coffee culture.